In [ ]:
!pip install -U langchain-ollama

In [1]:
import ollama
from langchain_ollama import OllamaLLM
from langchain.chains import create_retrieval_chain  
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import PromptTemplate

KeyboardInterrupt: 

In [2]:
model_name = "zephyr:7b-beta-q4_K_S"
llm = OllamaLLM(model=model_name, max_tokens=200) 

In [3]:
persist_directory = "../model/taxadvisordb_v2-0"

In [4]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [5]:
vector_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [6]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

In [7]:
docs = retriever.invoke("Can failing to submit an income tax return lead to prosecution?")
print(docs)

[Document(id='c4957c7b-e08c-4a87-86eb-4e1cd9eb6acf', metadata={'keywords': 'taxable income, income tax return, file taxes, non-filing, tax consequences, penalties, IRS', 'source': 'ITR_FAQs.csv'}, page_content="Query: What will happen when I have taxable income but do not File my Income Tax Return?\nSolution: Having taxable income and not filing an income tax return can put you in trouble with the Income Tax Authorities. Be prepared to welcome notices from the Income Tax Department along with fines and penalties. The consequences of non-filing of return are:\n\nLate filing Fee u/s 234F shall be levied if the return is not filed on or before the due date. The quantum of fees shall vary according to the amount of your income and return filing date. The maximum fee levied is Rs. 5000/-.\nNotice u/s 142(1), which requires you to file your return if you have not filed it before the due date. if you fail to comply with this notice, then it may result in\n(a) Best Judgment Assessment u/s 144 

In [8]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [9]:
prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template=(
        "You are a tax expert. Use the following information to answer the user's question:\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n\n"
        "Answer:"
    ),
)

In [10]:
import asyncio

async def get_answer(query):
    retriever = vector_db.as_retriever(search_kwargs={"k": 2})  # Fetch fewer docs
    docs = await asyncio.to_thread(retriever.invoke, query)  # Async retrieval

    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = f"You are a tax expert. Use the following information to answer the user's question:\n\nContext: {context}\n\nQuery: {query}\n\nAnswer:"

    # Stream response from LLM
    async for chunk in llm.astream(prompt):  
        print(chunk, end="", flush=True)

In [ ]:
query = "what is apple"
await get_answer(query)

Apple refers to a multinational technology company headquartered in Cupertino, California. It designs, manufactures, and sells electronic devices, computer software, and online services, most notably the iPhone, iPad, Mac lineup, Apple Watch, and various accessories, software, and services related to these products. As of 2021, Apple is one of the world's largest technology companies by revenue and market capitalization.

In terms of taxation, Apple has faced criticism over its complex global tax structure, which involves routing profits through low-tax jurisdictions like Ireland and the British Virgin Islands to minimize its overall tax bill. The company has also been accused of avoiding taxes through transfer pricing practices that shift profits to subsidiaries in lower-tax countries. These issues have led to investigations by governments around the world, including the United States, Europe, and Asia, as well as calls for increased transparency and reforms to international tax rules

: 